Created on 4/2/2020
Author: Yuan-Chi Yang

Objective: template for content analysis on political feedback, please feel free to modify and play around

In [168]:
import pandas as pd
import nltk
import re

# Importing the data and perform some checks

The data is from "C:\Users\yyang60\PostDoct-Emory\medicaid-project\medicaid-classifier\labeling-data\whole_dataset\BERT_20200228\political-tweets-streaming.csv"

It consists of all the tweets classified as the 'p' class by the best performing classfiers to date.

In [169]:
df = pd.read_csv('./political-tweets-streaming.csv',header = 0, keep_default_na=False,dtype={'tweet_id':str})

In [170]:
len(df)

383969

In [171]:
df.columns

Index(['tweet_id', 'unprocessed_text', 'class'], dtype='object')

### Check Duplicates
No Duplicated

In [173]:
df.duplicated(subset = ['tweet_id'], keep=False).sum()

0

### Include 'text_remove_stopwords' column

In [174]:
def loadStopWords(FILENAME):
    stopword_list = []
    infile = open(FILENAME)
    for line in infile:
        stopword_list.append(line.strip())
    print(len(stopword_list))
    return stopword_list

In [175]:
def processing_text_remove_stopwords(tweet_text,stop_words):
    tweet_text = re.sub(r'&amp;', "and", tweet_text)
    tweet_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',
                        tweet_text)
    #tweet_text = re.sub('(@[A-Za-z0-9\_]+)', '', tweet_text)
    tweet_text = re.sub('(@[\S]+)', '', tweet_text)#added Angel
    ####the following section separate words following each hashtag
    list_hashtag1=re.findall('(#[\S]+)',tweet_text)
    list_hashtag2=[]
    for p in list_hashtag1:
        p=re.sub('(#)',' ',p)#clear hashtag symbols
        p1=re.findall('([A-Z]{2,})',p)#uppercase abbrevs
        for p2 in p1:
            list_hashtag2.append(p2)#add abbrevs
        p=re.sub('([A-Z]{2,})',' ',p)#clear uppercase abbrevs
        p1=re.findall('([A-Z][a-z]{1,})',p)#words start uppercase letter
        for p2 in p1:
            list_hashtag2.append(p2)
        p=re.sub('([A-Z][a-z]{1,})',' ',p)#clear words start with uppercase letter
        p=re.sub('([A-Z])',' ',p)#clear single uppercase letters
        p=re.sub('([^A-Za-z])',' ',p)#clear symbols
        p1=p.split()#find leftover words
        for p2 in p1:
            list_hashtag2.append(p2)#add leftover words
    tweet_text=re.sub('(#[\S]+)','',tweet_text)
    
    tweet_text = re.sub("[^a-zA-Z_-]", " ", tweet_text)
    tweet_text = tweet_text.lower()
    tweet_text = re.sub(r'\s{2,}', " ", tweet_text)
    list_hashtag= [h for h in list_hashtag2 if (not h in stop_words and len(h)>1)]
    tweet_text = [t for t in tweet_text.split() if (not t in stop_words and len(t)>1)]
    tweet_text.extend(list_hashtag)
    return ' '.join(tweet_text)

In [176]:
stopwords = set(loadStopWords('./stopwords.txt'))

261


In [177]:
df['text_remove_stopwords'] = df['unprocessed_text'].apply(lambda x: processing_text_remove_stopwords(x,stopwords))

## Find the most frequent words in text

In [178]:
unigrams = ' '.join(df['text_remove_stopwords'].to_list()).split()
bigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    bigrams += list(nltk.bigrams(tweet.split()))
    
trigrams = []
for tweet in df['text_remove_stopwords'].to_list():
    trigrams += list(nltk.trigrams(tweet.split()))
uni_fd = nltk.FreqDist(unigrams)
big_fd = nltk.FreqDist(bigrams)
trig_fd = nltk.FreqDist(trigrams)


In [181]:
def wordInNumTweets(s):
    pattern = rf'(^|[^a-zA-Z]){s}([^a-zA-Z]|$)' #rf is for using a variable inside
    count=0
    for i in range(len(df['text_remove_stopwords'])):
         if re.search(pattern,df['text_remove_stopwords'].iloc[i]) is not None :
            count=count+1
    return count

In [182]:
num = 40
print('unigrams:\n')
fd = uni_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
df_ubt=[]
for i in range(0,num):
    print(fd_list[i][0], fd_list[i][1])
    df_ubt['uni_term']=fd_list[i][0]
    df_ubt['uni_count']=fd_list[i][1]
    df_ubt['uni_num_of_tweets']=wordInNumTweets(fd_list[i][0])

print('--------------------------------------------------------\n')
    
print('bigrams:\n')
fd = big_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
for i in range(0,num):
    x, y= fd_list[i][0]
    term = x + ' '+ y
    print(term, fd_list[i][1])
    df_ubt['bi_term']=term
    df_ubt['bi_count']=fd_list[i][1]
    df_ubt['bi_num_of_tweets']=wordInNumTweets(term)

print('--------------------------------------------------------\n')
print('trigrams:\n')

fd = trig_fd
fd_list = [(x,fd[x]) for x in fd]
fd_list.sort(key = lambda x: x[1], reverse = True)
for i in range(0,num):
    x, y, z= fd_list[i][0]
    term = x + ' '+ y + ' ' + z
    print(term, fd_list[i][1])
    df_ubt['tri_term']=term
    df_ubt['tri_count']=fd_list[i][1]
    df_ubt['tri_num_of_tweets']=wordInNumTweets(term)

df_ubt.to_csv('uni_bi_tri.csv')

SyntaxError: invalid syntax (<ipython-input-182-3998600fbf1c>, line 11)

## Finding the interesting terms

In [71]:
def highfreqword(text,term):
    pattern = rf'(^|[^a-zA-Z]){term}([^a-zA-Z]|$)' #rf is for using a variable inside
    #pattern = rf'{term}'
    if(re.search(pattern,text)!=None):
        return 1
    else:
        return 0

In [128]:
bigrams_ls=['social security','mental health','healthcare','insurance','food stamps','health care','health insurance','private insurance','cuts social','take away','work requirements','trump budget','pre-existing conditions','insurance companies','expansion','budget cuts']
for i in range(16):
    coln=bigrams_ls[i]
    df[coln] = df['text_remove_stopwords'].apply(lambda x:highfreqword(x,coln))

In [129]:
df_temp=df[df['middle class'].apply(lambda x:x==1)]
len(df_temp)

3382

In [132]:
for i in range(16):
    df_temp1=df_temp[df[bigrams_ls[i]].apply(lambda x:x==1)]
    print(len(df_temp1))
df_temp1=df_temp[df['healthcare'].apply(lambda x:x==1)]

<ipython-input-132-7e03087d1343>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp1=df_temp[df[bigrams_ls[i]].apply(lambda x:x==1)]
<ipython-input-132-7e03087d1343>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp1=df_temp[df['healthcare'].apply(lambda x:x==1)]


862
11
458
385
125
157
100
47
108
75
8
31
43
18
144
28


In [54]:
#term2 = 'trump budget'
#pattern = rf'(^|[^a-zA-Z]){term2}([^a-zA-Z]|$)' #rf is for using a variable inside
#df_temp2 = df[df['text_remove_stopwords'].apply(lambda x: re.search(pattern,x)!=None)]
#print('# of tweets:',len(df_temp2))


In [55]:
#df_temp12 =df_temp1.append(df_temp2)
#print('# of duplicated tweets:',df_temp12.duplicated(subset = ['tweet_id'], keep=False).sum())

# of duplicated tweets: 0


In [136]:
num = 30
temp = df_temp1.sample(n=num)
#for i in range(0,num):
    #print(temp['unprocessed_text'].iloc[i],'\n\n')
    #print(temp['text_remove_stopwords'].iloc[i],'\n\n')
    #print('---------------------------------------------------------------------------------')

In [137]:
#temp.to_csv('ss_pec_sample.csv')
#temp.to_csv('mh_e_sample.csv')
temp.to_csv('mc_h_sample.csv')

